In [ ]:
import copy

import numpy as np
from keras.layers import Input, Convolution2D, Dense, Dropout, Flatten, concatenate, BatchNormalization
from keras.models import Model  # basic class for specifying and training a neural network
from keras import losses
import keras
import tensorflow as tf
from sortedcontainers import SortedSet

random_state = np.random.RandomState(42)

MIN_Q = -1
MAX_Q = 1

In [ ]:

from collections import defaultdict

import numpy as np
import random

class Move:
    def __init__(self, player, x, y):
        self.player = player
        self.x = x
        self.y = y


def peek_stack(list):
    if len(list) == 0:
        return None
    else:
        return list[-1]
    
class GameState():
    WON = 1
    DRAW = 2
    NOT_OVER = 3

class Board:
    player_to_move: int
    NO_PLAYER = 0
    FIRST_PLAYER = 1
    SECOND_PLAYER = -1

    def __init__(self, size=5, win_chain_length=3):
        self.size = size
        self.matrix = np.zeros((self.size, self.size))
        self.matrix.fill(Board.NO_PLAYER)
        self.win_chain_length = win_chain_length
        # store (x, y, player) tuple
        self.ops = []
        self.player_to_move = 1
        self.game_state = GameState.NOT_OVER
        self.available_moves = set()
        for i in range(self.size):
            for j in range(self.size):
                self.available_moves.add((i, j))

        self.cached_point_rotations = defaultdict(list)
        self.cache_rotations()

        # whether current game_state is accurate
        self.state_computed = False

    def _mark_not_computed(self):
        self.state_computed = False

    # lightweight version of move
    def hypothetical_move(self, x, y):
        assert (self.game_state is GameState.NOT_OVER)
        self.ops.append(Move(self.player_to_move, x, y))
        self.matrix[x, y] = self.player_to_move
        self.available_moves.remove((x, y))
        self.flip_player_to_move()
        self._mark_not_computed()

    def unmove(self):
        previous_move = self.ops.pop()
        self.matrix[previous_move.x, previous_move.y] = Board.NO_PLAYER
        self.available_moves.add((previous_move.x, previous_move.y))
        self.flip_player_to_move()
        self.game_state = GameState.NOT_OVER

    # +1 for self, -1 for other
    def get_matrix(self, as_player):
        if as_player == Board.FIRST_PLAYER:
            return np.copy(self.matrix)
        return -np.copy(self.matrix)

    def get_rotated_matrices(self, as_player):
        matrix = self.get_matrix(as_player)
        return [
            matrix,
            matrix.transpose(),
            np.rot90(matrix),
            np.rot90(matrix).transpose(),
            np.rot90(matrix, 2),
            np.rot90(matrix, 2).transpose(),
            np.rot90(matrix, 3),
            np.rot90(matrix, 3).transpose()
        ]

    def cache_rotations(self):
        indices = np.array(range(self.size ** 2)).reshape(self.size, self.size)
        for matrix in [
                indices,
                indices.transpose(),
                np.rot90(indices),
                np.rot90(indices).transpose(),
                np.rot90(indices, 2),
                np.rot90(indices, 2).transpose(),
                np.rot90(indices, 3),
                np.rot90(indices, 3).transpose()
            ]:
            for x in range(self.size):
                for y in range(self.size):
                    self.cached_point_rotations[matrix[x, y]].append(indices[x, y])


    def coordinate_to_index(self, x, y):
        return x * self.size + y

    def get_rotated_point(self, index):
        return self.cached_point_rotations[index]

    # deprecate
    def make_move(self, x, y):
        assert(self.game_state is GameState.NOT_OVER)
        self.ops.append(Move(self.player_to_move, x, y))
        self.matrix[x, y] = self.player_to_move
        self.available_moves.remove((x, y))
        self.flip_player_to_move()
        self.compute_game_state()

    def make_random_move(self):
        move_x, move_y = random.choice(list(self.available_moves))
        self.hypothetical_move(move_x, move_y)

    def flip_player_to_move(self):
        if self.player_to_move == Board.FIRST_PLAYER:
            self.player_to_move = Board.SECOND_PLAYER
        else:
            self.player_to_move = Board.FIRST_PLAYER

    # returns None if game has not concluded, True if the last move won the game, False if draw
    # frequently called function, needs to be optimized
    def compute_game_state(self):
        last_move = peek_stack(self.ops)
        if last_move:
            last_x, last_y = last_move.x, last_move.y
            if self.chain_length(last_x, last_y, -1, 0) + self.chain_length(last_x, last_y, 1, 0) >= self.win_chain_length + 1\
                    or self.chain_length(last_x, last_y, -1, 1) + self.chain_length(last_x, last_y, 1, -1) >= self.win_chain_length + 1\
                    or self.chain_length(last_x, last_y, 1, 1) + self.chain_length(last_x, last_y, -1, -1) >= self.win_chain_length + 1\
                    or self.chain_length(last_x, last_y, 0, 1) + self.chain_length(last_x, last_y, 0, -1) >= self.win_chain_length + 1:
                self.game_state = GameState.WON
                return
            if len(self.ops) == self.size ** 2:
                self.game_state = GameState.DRAW
                return
        self.game_state = GameState.NOT_OVER
        self.state_computed = True

    def in_bounds(self, x, y):
        return 0 <= x < self.size and 0 <= y < self.size

    def chain_length(self, center_x, center_y, delta_x, delta_y):
        center_stone = self.matrix[center_x, center_y]
        if center_stone == Board.NO_PLAYER:
            return 0
        chain_length = 1
        for step in range(1, self.win_chain_length):
            step_x = delta_x * step
            step_y = delta_y * step
            if 0 <= center_x + step_x < self.size and 0 <= center_y + step_y < self.size and \
                    self.matrix[center_x + step_x, center_y + step_y] == center_stone:
                chain_length += 1
            else:
                break
        return chain_length

    # runs a full compute
    def game_won(self):
        if not self.state_computed:
            self.compute_game_state()
        return self.game_state == GameState.WON

    # probably drawn, cheap check
    def game_drawn(self):
        return len(self.ops) == self.size ** 2

    def game_over(self):
        if not self.state_computed:
            self.compute_game_state()
        return self.game_state != GameState.NOT_OVER

    def pprint(self):
        def display_char(x, y):
            move = peek_stack(self.ops)
            if move:
                was_last_move = (x == move.x and y == move.y)
                if self.matrix[x, y] == Board.FIRST_PLAYER:
                    if was_last_move:
                        return 'X'
                    return 'x'
                elif self.matrix[x, y] == Board.SECOND_PLAYER:
                    if was_last_move:
                        return 'O'
                    return 'o'
            return ' '
        board_string = ""
        for i in range(0, self.size):
            board_string += "\n"
            for j in range(self.size):
                board_string += "|" + display_char(j, i)
            board_string += "|"
        return board_string

    def __str__(self):
        return self.pprint()



In [ ]:
import numpy as np

class MoveList:
    # moves should be a tuple
    def __init__(self, moves):
        self.moves = moves
        # for move in moves:
        #    assert(len(move) == 2)

    def append(self, new_move):
        return MoveList((self.moves + (new_move,)))

    def __eq__(self, other):
        return self.moves == other.moves

    def __hash__(self):
        return hash(self.moves)

    def __len__(self):
        return len(self.moves)

# principle value search
class PVSNode:
    MAX_Q = 1
    MIN_Q = -1

    def __init__(self, parent, is_maximizing, full_move_list):

        # parent TreeNode
        self.parent = parent
        # full move tree
        self.full_move_list = full_move_list

        self.is_maximizing = is_maximizing
        # is the game over with this move list
        self.game_status = GameState.NOT_OVER
        # for debugging
        # key is move
        self.children = {}

        # tuple of PVSNode, q
        self.principle_variation = None
        self.q = None

        # log likelihood of playing this move given parent state
        self.log_local_p = 0
        # log likelihood of playing this move given root board state (used for PVS search)
        self.log_total_p = 0

        # the quality of this move
        self.move_goodness = 0

    def has_children(self):
        return len(self.children) > 0

    # note this does NOT compute q for child
    def create_child(self, move):
        child = PVSNode(parent=self,
                        is_maximizing=not self.is_maximizing,
                        full_move_list=self.full_move_list.append(move))
        self.children[move] = child
        return child

    def get_sorted_moves(self):
        return sorted(self.children.items(), key=lambda x: x[1].move_goodness, reverse=self.is_maximizing)

    # used ONLY for leaf assignment
    def assign_q(self, q, game_status):
        assert (len(self.children) == 0)

        # mark game status
        self.game_status = game_status

        # for a leaf node, the principle variation is itself
        self.principle_variation = self
        self.q = q

        self.assign_move_goodness()

    def assign_move_goodness(self):
        # play shorter sequences if advantageous, otherwise play longer sequences
        if self.q > 0:
            self.move_goodness = self.q - len(self.principle_variation.full_move_list) * 0.001
        else:
            self.move_goodness = self.q + len(self.principle_variation.full_move_list) * 0.001

    def assign_p(self, log_p):
        self.log_local_p = log_p
        self.log_total_p = self.parent.log_total_p + self.log_local_p

    def recalculate_q(self):
        # take the largest (or smallest) q across all seen moves

        # if this node is still a leaf, break
        if len(self.children) == 0:
            return

        if self.principle_variation:
            prev_q = self.q
        else:
            prev_q = np.inf

        if self.is_maximizing:
            best_move = max(self.children.items(), key=lambda x : x[1].move_goodness)[0]
        else:
            best_move = min(self.children.items(), key=lambda x : x[1].move_goodness)[0]

        # using negamax framework
        self.principle_variation = self.children[best_move].principle_variation
        self.q = self.children[best_move].q

        self.assign_move_goodness()

        if self.parent and abs(prev_q - self.q) > 1E-6:
            # update pvs for parent
            self.parent.recalculate_q()

    def __str__(self):
        if self.principle_variation:
            return ("PV: " + str(self.principle_variation.full_move_list.moves) + " Q: {0:.4f} P: {1:.4f}").format(self.q, self.log_total_p)
        else:
            return "Position: " + str(self.full_move_list.moves)


In [ ]:


class PQMind:
    def __init__(self, size, alpha, turn_input=True, init=True):

        self.size = size

        self.value_est = self.get_value_model()
        self.policy_est = self.get_policy_model()

        # initialization
        init_examples = 10

        if init:
            sample_x = [
                                    random_state.random_integers(-1, 1, size=(init_examples, size, size, 1)),
                                    random_state.random_integers(-1, 1, size=(init_examples)).reshape(init_examples, -1),
                                ]
            self.value_est.fit(sample_x, y=np.zeros((init_examples)), epochs=1, batch_size=100)
            self.policy_est.fit(sample_x, y=np.zeros((init_examples, self.size ** 2)))

        self.train_vectors = []
        self.train_q = []
        self.train_p = []

        self.fitted = False

        self.turn_input = turn_input

        self.alpha = alpha

    def get_layers(self):
        inp = Input(shape=(self.size, self.size, 1))

        #bn1 = BatchNormalization()(inp)
        # key difference between this and conv network is padding
        conv_1 = Convolution2D(64, (3, 3), padding='same', activation='relu',
                               kernel_initializer='random_uniform')(inp)
        bn2 = BatchNormalization()(conv_1)
        conv_2 = Convolution2D(64, (3, 3), padding='same', activation='relu',
                               kernel_initializer='random_uniform')(bn2)
        bn3 = BatchNormalization()(conv_2)
        conv_3 = Convolution2D(64, (3, 3), padding='valid', activation='relu',
                               kernel_initializer='random_uniform')(bn3)
        bn4 = BatchNormalization()(conv_3)

        flat = Flatten()(bn4)
        turn_input = Input(shape=(1,), name='turn')
        full = concatenate([flat, turn_input])

        hidden = Dense(30, activation='relu', kernel_initializer='random_uniform')(full)
        #bn4 = BatchNormalization()(hidden)

        return inp, turn_input, hidden

    def get_value_model(self):
        inp, turn_input, hidden = self.get_layers()

        out = Dense(1)(hidden)

        model = Model(inputs=[inp, turn_input], outputs=out)
        model.compile(loss=losses.mean_squared_error, optimizer='adam', metrics=['mean_squared_error'])

        return model

    def get_policy_model(self):
        inp, turn_input, hidden = self.get_layers()


        out = Dense(self.size ** 2, activation='softmax')(hidden)

        model = Model(inputs=[inp, turn_input], outputs=out)
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

        return model

    # after parents are expanded, this method recomputes all leaves
    def pvs_catch_leaves(self, leaf_nodes, new_parents, max_depth=5):
        for parent in new_parents:
            if parent in leaf_nodes:
                leaf_nodes.remove(parent)
            if len(parent.full_move_list) < max_depth:
                leaf_nodes.update([node for node in parent.children.values() if node.game_status == GameState.NOT_OVER])

    def pvs_k_principle_variations(self, root_node, leaf_nodes, k=5):
        # include the best move according to q
        q_best = root_node.principle_variation
        # draw
        if q_best is None:
            return None

        # include k-1 most likely moves according to p
        if q_best.game_status == GameState.NOT_OVER:
            return [q_best] + list(leaf_nodes.islice(0, k - 2))
        else:
            return leaf_nodes.islice(0, k - 1)

    # board perception AND move turn perception will always be from the perspective of Player 1
    # Q will always be from the perspective of Player 1 (Player 1 Wins = Q = 1, Player -1 Wins, Q = -1)

    def pvs_best_moves(self, board, max_iters=10, epsilon=0.01, verbose=True, k=25, max_depth=5):
        root_node = PVSNode(parent=None,
                                    is_maximizing=True if board.player_to_move == 1 else False,
                                    full_move_list=MoveList(moves=()))

        principle_variations = [root_node]
        leaf_nodes = SortedSet(principle_variations, key=lambda x: x.log_total_p)

        for i in range(max_iters):
            self.pvs_batch(board, principle_variations)
            self.pvs_catch_leaves(leaf_nodes, principle_variations, max_depth=max_depth)
            principle_variations = self.pvs_k_principle_variations(root_node, leaf_nodes, k=k)

            if not principle_variations:
                print("Exhausted Search")
                break

        # find best node (highest q)
        possible_moves = root_node.get_sorted_moves()

        for move, q in possible_moves:
            node = root_node.children[move]
            print(str(node.principle_variation))

        return possible_moves

    def pvs(self, board, max_iters=10, epsilon=0.01, verbose=True, max_depth=5, k=25):

        # array of [best_move, best_node]
        possible_moves = self.pvs_best_moves(board,
                                             max_iters=max_iters,
                                             epsilon=epsilon,
                                             verbose=verbose,
                                             k=k,
                                             max_depth=max_depth)

        # best action is 0th index
        picked_action = 0

        # pick a suboptimal move
        if random_state.random_sample() < epsilon:
            if verbose:
                print('suboptimal move')
            # abs is only there to handle floating point problems
            qs = np.array([node.q for _, node in possible_moves])
            if board.player_to_move == 1:
                distribution = np.abs(qs + 1.0) / 2
            else:
                # not sure this is correct
                distribution = sorted(-np.abs(qs - 1.0) / 2)

            if sum(distribution) > 0:
                distribution = (distribution.astype(np.float64) / sum(distribution))
                picked_action = np.random.choice(range(len(possible_moves)), 1, p=distribution)[0]

        best_move, best_node = possible_moves[picked_action]

        return best_move, best_node.q

    def pvs_vectors(self, board, nodes_to_expand):
        # each board state is defined by a list of moves
        q_search_nodes = []
        q_search_vectors = []
        q_search_player = []

        p_search_vectors = []
        p_search_players = []

        validation_matrix = np.copy(board.matrix)

        for parent in nodes_to_expand:
            # for each move except the last, make rapid moves on board
            for move in parent.full_move_list.moves:
                board.hypothetical_move(move[0], move[1])

            for child_move in copy.copy(board.available_moves):
                child = parent.create_child(child_move)
                board.hypothetical_move(child_move[0], child_move[1])
                # if game is over, then we have our q
                if board.game_won():
                    # the player who last move won!
                    if len(child.full_move_list.moves) == 1:
                        print('win now')
                    child.assign_q(-board.player_to_move, GameState.WON)

                elif board.game_drawn():
                    child.assign_q(0, GameState.DRAW)

                else:
                    q_search_nodes.append(child)
                    vector, player = board.get_matrix(as_player=board.player_to_move), board.player_to_move
                    q_search_vectors.append(vector)
                    q_search_player.append(player)

                # unmove for child
                board.unmove()

            # update p
            vector, player = board.get_matrix(as_player=board.player_to_move), board.player_to_move
            p_search_vectors.append(vector)
            p_search_players.append(player)

            # unwind parent
            for i in range(len(parent.full_move_list)):
                board.unmove()

        assert(np.array_equal(board.matrix, validation_matrix))

        return q_search_nodes, q_search_vectors, q_search_player, p_search_vectors, p_search_players

    def pvs_batch(self, board, nodes_to_expand):

        for parent in nodes_to_expand:
            for move in parent.full_move_list.moves:
                board.hypothetical_move(move[0], move[1])

            if board.game_won():
                print('game over??!')

            for move in parent.full_move_list.moves:
                board.unmove()

        q_search_nodes, q_search_vectors, q_search_player, p_search_vectors, p_search_players = \
            self.pvs_vectors(board, nodes_to_expand)

        if len(q_search_vectors) == 0:
            # recalculate for wins/draws
            for parent in nodes_to_expand:
                parent.recalculate_q()
            return False

        q_board_vectors = np.array(q_search_vectors).reshape(len(q_search_vectors), self.size, self.size, 1)
        p_board_vectors = np.array(p_search_vectors).reshape(len(p_search_vectors), self.size, self.size, 1)

        # this helps parallelize
        # multiplication is needed to flip the Q (adjust perspective)
        q_predictions = np.clip(
                    self.value_est.predict([q_board_vectors, np.array(q_search_player)],
                                                        batch_size=64).reshape(len(q_search_vectors)),
                    a_max=PVSNode.MAX_Q - 0.01,
                    a_min=PVSNode.MIN_Q + 0.01
            )

        log_p_predictions = np.log(self.policy_est.predict([p_board_vectors, np.array(p_search_players)],
                                                        batch_size=64).reshape((len(p_search_vectors), self.size ** 2)))

        for i, parent in enumerate(nodes_to_expand):
            for move in parent.children.keys():
                move_index = self.move_to_index(move)
                parent.children[move[0], move[1]].assign_p(log_p_predictions[i][move_index])

        for i, leaf in enumerate(q_search_nodes):
            # update with newly computed q's (only an assignment since approx, we'll compute minimax q's later)
            leaf.assign_q(q_predictions[i], GameState.NOT_OVER)

        # for all the nodes whose leaves' q's are calculated
        for parent in nodes_to_expand:
            parent.recalculate_q()

        return True

    def q(self, board, as_player):
        prediction = self.value_est.predict([np.array([board.get_matrix(as_player).reshape(board.size, board.size, -1)]), np.array([as_player])])[0][0]
        return prediction

    # with epsilon probability will select random move
    # returns whether game has concluded or not
    def make_move(self, board, as_player, retrain=True, verbose=True, epsilon=0.1, max_depth=5, max_iters=10, k=25):
        current_q = self.q(board, as_player)
        assert(as_player == board.player_to_move)

        move, best_q = self.pvs(board,
                                epsilon=epsilon,
                                verbose=verbose,
                                max_depth=max_depth,
                                max_iters=max_iters,
                                k=k)

        new_q = (1 - self.alpha) * current_q + self.alpha * best_q
        print(current_q, best_q)
        self.add_train_example(board, as_player, new_q, move)

        board.make_move(move[0], move[1])

        if board.game_over():
            return True

        return False

    def one_hot_p(self, move_index):
        vector = np.zeros((self.size ** 2))
        vector[move_index] = 1.
        return vector

    def move_to_index(self, move):
        return move[0] * self.size + move[1]

    def index_to_move(self, index):
        return np.int(index / self.size), index % self.size

    # adds rotations
    def add_train_example(self, board, as_player, result, move, invert_board=False):
        board_vectors = board.get_rotated_matrices(as_player=as_player)

        for i, vector in enumerate(board_vectors):
            clamped_result = max(min(result, MAX_Q), MIN_Q)
            self.train_vectors.append((vector, as_player))
            self.train_q.append(clamped_result)
            # get the i'th rotation
            which_rotation = board.get_rotated_point(self.move_to_index(move))[i]
            self.train_p.append(self.one_hot_p(which_rotation))
            #self.train_p.append(which_rotation)

    def update_model(self):

        train_inputs = [[], []]
        for vector, whose_move in self.train_vectors:
            train_inputs[0].append(vector.reshape(self.size, self.size, 1))
            train_inputs[1].append(whose_move)
            
        train_inputs[0] = np.array(train_inputs[0])
        train_inputs[1] = np.array(train_inputs[1])

        print(len(self.train_vectors))
        if len(self.train_vectors) > 0:
            self.value_est.fit(x=train_inputs,
                                y=np.array(self.train_q),
                                shuffle=True,
                                validation_split=0.1)
            self.policy_est.fit(x=train_inputs,
                                y=np.array(self.train_p),
                                shuffle=True,
                                validation_split=0.1)

        max_vectors = 100000
        while len(self.train_vectors) > max_vectors:
            self.train_vectors = self.train_vectors[100:]
            self.train_p = self.train_p[100:]
            self.train_q = self.train_q[100:]

        print('Num Train Vectors', len(self.train_vectors))

    def save(self, filename):
        self.value_est.save(filename + '_value.net')
        self.policy_est.save(filename + '_policy.net')

    def load_net(self, filename):
        self.value_est = keras.models.load_model(filename + '_value.net')
        self.policy_est = keras.models.load_model(filename + '_policy.net')
       
    def load(self, value_file, policy_file):
        self.value_est = keras.models.load_model(value_file)
        self.policy_est = keras.models.load_model(policy_file)

In [ ]:


def play_a_game():
    mind = PQMind(size=SIZE, alpha=0.2, turn_input=True, init=False)
    mind.value_est.set_weights(q_model_bc.value)
    mind.policy_est.set_weights(p_model_bc.value)
    round_board = Board(size=SIZE, win_chain_length=WIN_CHAIN_LENGTH)
    
    # randomize the board a bit
    for i in range(random.randint(0, 3)):
        round_board.make_random_move()
    
    current_player = round_board.player_to_move
    while True:
        result = mind.make_move(round_board,
                                as_player=current_player,
                                retrain=True,
                                epsilon=0.1,
                                max_depth=25,
                                k=SIZE ** 2,
                                max_iters=50,
                                )
        print(round_board.pprint())
        current_player = -current_player
        if result:
            break
            
    return mind.train_vectors, mind.train_p, mind.train_q

SIZE = 7
WIN_CHAIN_LENGTH = 5

mind = PQMind(size=SIZE, alpha=0.2, turn_input=True, init=True)
q_model = mind.value_est
p_model = mind.policy_est

def distributed_play():

    collected = sc.parallelize(range(400)).repartition(400).map(lambda x : play_a_game()).collect()

    train_vectors = []
    train_p = []
    train_q = []

    for vector, p, q in collected:
        train_vectors.extend(vector)
        train_p.extend(p)
        train_q.extend(q)


    train_inputs = [[], []]
    for vector, whose_move in train_vectors:
        train_inputs[0].append(vector.reshape(SIZE, SIZE, 1))
        train_inputs[1].append(whose_move)

    train_inputs[0] = np.array(train_inputs[0])
    train_inputs[1] = np.array(train_inputs[1])

    if len(train_vectors) > 0:
        q_model.fit(x=train_inputs,
                            y=np.array(train_q),
                            shuffle=True,
                            validation_split=0.1)
        p_model.fit(x=train_inputs,
                            y=np.array(train_p),
                            shuffle=True,
                            validation_split=0.1)

    print('Num Train Vectors', len(train_vectors))




In [ ]:
epochs = 500
for i in range(epochs):
    q_model_bc = sc.broadcast(copy.deepcopy(q_model.get_weights()))
    p_model_bc = sc.broadcast(copy.deepcopy(p_model.get_weights()))
    distributed_play()

In [ ]:
mind.value_est.set_weights(q_model.get_weights())
mind.policy_est.set_weights(p_model.get_weights())
mind.save('models/distributed_5_bn')